<a href="https://colab.research.google.com/github/garvit5555/Prime_factorisation_grover_algorithm/blob/main/grover_115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 36.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

def add_k_fourier(k, wires):
    """
    Apply controlled phase shift gate for Fourier addition.
    """
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

def multiplication(reg_x, reg_y, reg_z):
    """
    Simulate the multiplication of two registers reg_x and reg_y into reg_z using quantum Fourier transform.
    """
    # Prepare solution qubits for counting using QFT
    qml.QFT(wires=reg_z)

    # Add the product of reg_x and reg_y to the counter in reg_z
    for i in range(len(reg_y)):
        for j in range(len(reg_x)):
            coeff = 2 ** (len(reg_x) + len(reg_y) - i - j - 2)
            qml.ctrl(add_k_fourier, control=[reg_y[i], reg_x[j]])(coeff, reg_z)

    # Return to computational basis using inverse QFT
    qml.adjoint(qml.QFT)(wires=reg_z)

# Number we want to factorize (a biprime number)
N = int(input("Enter a number: "))

# Number of qubits needed to encode N
n = int(np.ceil(np.log2(N)))

# Define qubit registers for the circuit
reg_z = [i for i in range(n)]                # Qubits for the solution
reg_x = [i for i in range(n, 2*n - 2)]       # Qubits for encoding factor m
reg_y = [i for i in range(2*n - 2, 3*n - 3)] # Qubits for encoding factor k

# Define the quantum device
dev = qml.device("default.qubit", wires=reg_x + reg_y + reg_z)
n_wires = len(dev.wires)

# Create variable for the number of iterations of Grover's algorithm
number_of_iterations = int((np.pi / 4) * np.sqrt(21))  # Adjusted to 21 for better convergence

@qml.qnode(dev)


def factorization(N, reg_x, reg_y, reg_z, number_of_iterations):

    # Superposition of the input
    for wire in reg_x:
        qml.Hadamard(wires=wire)

    for wire in reg_y:
        qml.Hadamard(wires=wire)

    for i in range(number_of_iterations):
    # Apply the multiplication
      multiplication(reg_x, reg_y, reg_z)

    # Change sign of n
      qml.FlipSign(N, wires=reg_z)

    # Uncompute multiplication
      qml.adjoint(multiplication)(reg_x, reg_y, reg_z)

    # Apply Grover operator
      qml.GroverOperator(wires=reg_x + reg_y)

    return qml.probs(wires=reg_x)


# Visualize the quantum circuit
qml.draw_mpl(factorization, show_all_wires=True)(N, reg_x, reg_y, reg_z, 1)
plt.show()

p = np.argmax(factorization(N, reg_x, reg_y, reg_z, number_of_iterations))
q = np.argmax(factorization(N, reg_y, reg_x, reg_z, number_of_iterations))

if p>=q:
  p = N//q
else:
  q = N//p

print(f"Factors of {N} are: {q} and {p}")


Enter a number: 115


Factors of 115 are: 23 and 5


In [ ]:
#Just Tried For N=893

import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt

def add_k_fourier(k, wires):
    """
    Apply controlled phase shift gate for Fourier addition.
    """
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

def multiplication(reg_x, reg_y, reg_z):
    """
    Simulate the multiplication of two registers reg_x and reg_y into reg_z using quantum Fourier transform.
    """
    # Prepare solution qubits for counting using QFT
    qml.QFT(wires=reg_z)

    # Add the product of reg_x and reg_y to the counter in reg_z
    for i in range(len(reg_y)):
        for j in range(len(reg_x)):
            coeff = 2 ** (len(reg_x) + len(reg_y) - i - j - 2)
            qml.ctrl(add_k_fourier, control=[reg_y[i], reg_x[j]])(coeff, reg_z)

    # Return to computational basis using inverse QFT
    qml.adjoint(qml.QFT)(wires=reg_z)

# Number we want to factorize (a biprime number)
N = int(input("Enter a number: "))

# Number of qubits needed to encode N
n = int(np.ceil(np.log2(N)))

# Define qubit registers for the circuit
reg_z = [i for i in range(n)]                # Qubits for the solution
reg_x = [i for i in range(n, 2*n - 2)]       # Qubits for encoding factor m
reg_y = [i for i in range(2*n - 2, 3*n - 3)] # Qubits for encoding factor k

# Define the quantum device
dev = qml.device("default.qubit", wires=reg_x + reg_y + reg_z)
n_wires = len(dev.wires)

# Create variable for the number of iterations of Grover's algorithm
number_of_iterations = int((np.pi / 4) * np.sqrt(N))  # Adjusted to 21 for better convergence

@qml.qnode(dev)


def factorization(N, reg_x, reg_y, reg_z, number_of_iterations):

    # Superposition of the input
    for wire in reg_x:
        qml.Hadamard(wires=wire)

    for wire in reg_y:
        qml.Hadamard(wires=wire)

    for i in range(number_of_iterations):
    # Apply the multiplication
      multiplication(reg_x, reg_y, reg_z)

    # Change sign of n
      qml.FlipSign(N, wires=reg_z)

    # Uncompute multiplication
      qml.adjoint(multiplication)(reg_x, reg_y, reg_z)

    # Apply Grover operator
      qml.GroverOperator(wires=reg_x + reg_y)

    return qml.probs(wires=reg_x)




p = np.argmax(factorization(N, reg_x, reg_y, reg_z, number_of_iterations))
q = np.argmax(factorization(N, reg_y, reg_x, reg_z, number_of_iterations))

if p>=q:
  p = N//q
else:
  q = N//p

print(f"Factors of {N} are: {q} and {p}")


Enter a number: 893
